# Hito 1. Optimización evolutiva
- Algoritmo Genético Evolutivo Generacional
- Algoritmo Genético Evolutivo Generacional y Memético

### Miembros del grupo:
- Anny Álvarez Nogales
- Miguel González Álvarez
- Paula Arias Fernández
- Javier Quesada Pajares
- Jorge del Castillo Gómez
---

# Himmelblau - Optimización de números reales
Minimos globales teóricos de la funcion:
1. `(3.0, 2.0); f(x,y) = 0`
2. `(−2.805118,3.131312); f(x,y) = 0`
3. `(−3.779310,−3.283186); f(x,y) = 0`
4. `(3.584428,−1.848126); f(x,y) = 0`

In [ ]:
# Función himmelblau

import random

# Esta funcion recibe un solo cromosoma
onlyone = False

def himmelblau (ch):
	x = ch[0]
	y = ch[1]
	fxy = (x**2 + y - 11)**2 + (x + y**2 -7)**2
	if onlyone and (x>0 or y>0): # himmelblau modified to have only one global minimum (-3.77, -3.28)
		fxy += 0.5
	return fxy

In [ ]:
# fitness para himmelblau: valor mínimo de la función, a mayor valor de fitness, menor valor de la función
# maximizamos función

def fitness_himmel (ch):
	return 1 / (1 + himmelblau(ch))

## Prueba 1. AG - Evolución generacional

In [ ]:
# Define operadores de números reales

rang = (-5, 5) # al no hacerlo con clases, debemos definir el rango como variable global

'''
    Devuelve un individuo seleccionado por torneo. Se selecciona el individuo de mayor fitness sobre un 
    conjunto aleatorio equiprobable T.
'''
def select (pop, T): 
    tournament = random.sample(pop, T)
    # Selección por torneo: selecciona el individuo con el mayor fitness (es decir, el más bajo en valor de Himmelblau)  
    return max(tournament, key=lambda ind: fitness_himmel(ind)).copy()


def create (alphabet=None, N=100): 
    pop = []

    for _ in range(N):
        values = [random.uniform(-5, 5) for _ in range(2)]  
        pop.append(values)
    return pop


'''
    Función que ordena una poblacion segun el fitness.
    @return: Devuelve la poblacion ordenada por fitness, y los valores fitness de cada individuo de la poblacion.
'''
def sort_pop (pop, fitness): 
    pop_with_fitness = [(indiv, fitness(indiv)) for indiv in pop]
    sorted_pop = sorted(pop_with_fitness, key=lambda x: x[1], reverse=True)  # Mayor fitness primero
    return [indiv for indiv, _ in sorted_pop], [fit for _, fit in sorted_pop]

'''
    Funcion que implementa el operador crossover: emparejamiento de dos individuos. pcross: probabilidad de que 
    se produzca el emparejamiento. Se implementa el emparejamiento para números reales basado en una combinacion
    de los genes de ambos padres. Los genes de cada padre tienen una mayor representacion en uno de los dos hijos.
    @return: dos hijos
'''
def crossover (ind1, ind2, pcross): # devuelve el cruce (emparejamiento) de dos individuos
    
    if (random.random() > pcross):
        return ind1.copy(), ind2.copy()

    beta = random.uniform(0,1)
    
    child1_x = beta*ind1[0] + (1 - beta)*ind2[0]
    child1_y = beta*ind1[1] + (1 - beta)*ind2[1]

    child2_x = (1 - beta)*ind1[0] + beta*ind2[0]
    child2_y = (1 - beta)*ind1[1] + beta*ind2[1]


    return [child1_x, child1_y], [child2_x, child2_y]


'''
    Función que muta un individuo. pmut: probabilidad de que se produzca la mutacion
    Este tipo de mutación reemplaza uno de los valores en el individuo con un nuevo valor generado
    aleatoriamente dentro de los límites definidos (rang).
'''
def mutate(ind, pmut):
    if random.random() < pmut:
        ind[random.randint(0, 1)] = random.uniform(rang[0], rang[1]) # otro tipo de mutacion
    return ind.copy()


'''
    Algoritmo de evolución generacional. Funcionamiento:
    1. Ordena la población inicial según su fitness
    2. Itera a través de un número dado de generaciones (ngen)
    3. En cada generación:
        - Si hay elitismo, guarda el mejor individuo directamente en la nueva población
        - Realiza selección por torneo para elegir padres
        - Aplica cruce con probabilidad 'pcross' para generar descendientes
        - Aplica mutación con probabilidad 'pmut' a los descendientes
        - Llena la nueva población con los descendientes generados
    4. Ordena la nueva población y evalúa el fitness
    5. Imprime el mejor fitness cada 'trace' generaciones, si está habilitado
    6. Devuelve la mejor población al final de las generaciones
    @return: poblacion final
'''
def evolve (pop, fit, pmut, ngen=100, T=2, trace=0, pcross=0.7, elitism=False):

    pop, _ = sort_pop(pop, fit)

    for i in range(ngen):
        new_poblacion = []

        if(elitism):
            best_indv = pop[0]
            new_poblacion.append(best_indv.copy())
            
        while len(new_poblacion) < len(pop):
            parent_1 = select(pop, T)
            parent_2 = select(pop, T)

            child_1, child_2 = crossover(parent_1, parent_2, pcross)

            child_1 = mutate(child_1, pmut)
            child_2 = mutate(child_2, pmut)

            new_poblacion.extend([child_1, child_2])
        
        pop, fitness = sort_pop(new_poblacion[:len(pop)], fit)

        if i % trace == 0 and trace > 0:
            print(f"Generacion {i}: mejor fitness [{fitness[0]}]")
    
    print(f"Generacion {ngen}: mejor fitness [{fitness[0]}]")

    return pop

### Decisiones

**Operadores básicos**:
- Selección: por torneo
- Emparejamiento: mezclado lineal 
- Mutación: por rango [vmin, vmax]

**Hiperparámetros**:
* *pmut*: 10/100
* *ngen*: 100
* T: 4
* *pcross*: 0.7
* elisitm: False

In [ ]:
# crea y evoluiona
pop = create()
pop = evolve(pop, fitness_himmel, pmut=10/100, ngen=100, T=4, trace=25, pcross=0.7, elitism=False)

Generacion 0: mejor fitness [0.2952568997016495]
Generacion 25: mejor fitness [0.838982892859534]
Generacion 50: mejor fitness [0.838982892859524]
Generacion 75: mejor fitness [0.8389828928595396]
Generacion 100: mejor fitness [0.8389828928595421]


In [14]:
# Mejor individuo, valor en la función y su fitness
best_individual = pop[0]  
himmel_value = himmelblau(best_individual)

print(f"El mejor individuo es {best_individual}")
print(f"Valor de la función de Himmelblau en ese individuo: {himmel_value:.10f}")
print(f"Fitness de la mejor solución: {fitness_himmel(best_individual)}")


El mejor individuo es [3.531729179447823, -1.77507053290786]
Valor de la función de Himmelblau en ese individuo: 0.1919194164
Fitness de la mejor solución: 0.8389828928595421


In [16]:
#Comprobación de valores en mínimos globales
print('Valores de fitness (ideal: 1), himmelblau (ideal: 0)')
print(f'Punto [3.0, 2.0]: {fitness_himmel([3.0, 2.0]), himmelblau([3.0, 2.0])}')
print(f'Punto [-2.8, 3.13] {fitness_himmel([-2.8, 3.13]), himmelblau([-2.8, 3.13])}')
print(f'Punto [-3.7, -3.28] {fitness_himmel([-3.7, -3.28]), himmelblau([-3.7, -3.28])}')
print(f'Punto [3.58, -1.84] {fitness_himmel([3.58, -1.84]), himmelblau([3.58, -1.84])}')

Valores de fitness (ideal: 1), himmelblau (ideal: 0)
Punto [3.0, 2.0]: (1.0, 0.0)
Punto [-2.8, 3.13] (0.9990912166384334, 0.0009096100000000729)
Punto [-3.7, -3.28] (0.7399128276141638, 0.35151055999999753)
Punto [3.58, -1.84] (0.9982627034519286, 0.0017403199999999868)


## Prueba 2. AG - Evolución generacional **memético**. Búsqueda local con gradiente.

Los algoritmos meméticos incorporan a un genético una fase de búsqueda local, posterior a la mutación. La búsqueda local se aplica después de la mutación y el cruce de los padres. Solo algunos individuos tienen la búsqueda local aplicada (probabilísticamente).

> Fases: [selección - emparejamiento - mutación - **Búsqueda local**].


In [ ]:
'''Se ejecuta la búsqueda local sobre los hijos mutados'''

"""
    Aplica una búsqueda local a un individuo utilizando el método de descenso por el gradiente.

    Parámetros:
        individual: individuo que será optimizado
        step_size: tamaño del paso para cada iteración del descenso por el gradiente. Valor predeterminado es 0.1.
        max_iterations: número máximo de iteraciones para la búsqueda local. Valor predeterminado es 1.

    @return:
        El individuo modificado después de aplicar la búsqueda local
"""
def local_search(individual, step_size=0.1, max_iterations=1):
    x, y = individual.copy()
    for _ in range(max_iterations):
        # Derivadas parciales de la función Himmelblau
        grad_x = 4*x*(x**2 + y - 11) + 2*(x + y**2 - 7)
        grad_y = 2*(x**2 + y - 11) + 4*y*(x + y**2 - 7)
        
        # Nuevos valores del individuo
        x -= step_size * grad_x
        y -= step_size * grad_y

        x = max(rang[0], min(rang[1], x))
        y = max(rang[0], min(rang[1], y))
    
    return [x, y] 


'''
Ejecuta la evolución generacional de una población utilizando selección, crossover, mutación y búsqueda local.
    Nuevos parámetros:
        step_size: Tamaño del paso para búsqueda local.
        local_search_prob: Probabilidad de aplicar búsqueda local.
        ls_function: Función de búsqueda local, por si se quiere cambiar.
'''
def evolve(pop, fit, pmut, ngen=100, T=2, trace=0, pcross=0.7, elitism=False, 
           step_size=0.1, local_search_prob=0.1, ls_function=None):

    pop, _ = sort_pop(pop, fit)

    for i in range(ngen):
        new_poblacion = []

        if elitism:
            best_indv = pop[0]
            new_poblacion.append(best_indv.copy())
            
        while len(new_poblacion) < len(pop):
            parent_1 = select(pop, T)
            parent_2 = select(pop, T)

            child_1, child_2 = crossover(parent_1, parent_2, pcross)

            child_1 = mutate(child_1, pmut)
            child_2 = mutate(child_2, pmut)

            # Nuevo: Búsqueda local
            if ls_function and random.random() < local_search_prob:
                child_1 = ls_function(child_1, step_size)
            if ls_function and random.random() < local_search_prob:
                child_2 = ls_function(child_2, step_size)

            new_poblacion.extend([child_1, child_2])
        
        pop, fitness = sort_pop(new_poblacion[:len(pop)], fit)

        if i % trace == 0 and trace > 0:
            print(f"Generacion {i}: mejor fitness [{fitness[0]}]")
    
    print(f"Generacion {ngen}: mejor fitness [{fitness[0]}]")

    return pop

In [35]:
#Visualizamos el resultado
population = create()
population = evolve(population, fit=fitness_himmel, pmut=0.1, ngen=1000, T=3, trace=100,elitism=True, step_size=0.1, local_search_prob=0.1, ls_function=local_search)

best_individual=population[0]
himmel_value = himmelblau(best_individual)

print(f"\nEl mejor individuo es {best_individual}")
print(f"Valor de la función de Himmelblau en ese individuo: {himmel_value:.10f}")
print(f"Valor de la función de Himmelblau: {himmelblau(best_individual)}")

Generacion 0: mejor fitness [0.714056094744559]
Generacion 100: mejor fitness [1.0]
Generacion 200: mejor fitness [1.0]
Generacion 300: mejor fitness [1.0]
Generacion 400: mejor fitness [1.0]
Generacion 500: mejor fitness [1.0]
Generacion 600: mejor fitness [1.0]
Generacion 700: mejor fitness [1.0]
Generacion 800: mejor fitness [1.0]
Generacion 900: mejor fitness [1.0]
Generacion 1000: mejor fitness [1.0]

El mejor individuo es [3.5844283409224182, -1.848126525747793]
Valor de la función de Himmelblau en ese individuo: 0.0000000000
Valor de la función de Himmelblau: 4.506093217087719e-17


## Prueba 3. AG - Evolución generacional memético - **Mutación con gradiente**
Se le considera un algoritmo memético porque incorpora un proceso de mejora local mediante el gradiente de la función. De tal forma, la mutación no es aleatoria, sino que está dirigida por el gradiente, ajustando los individuos eficientemente. 

In [36]:
'''PRUEBA EN MUTACIÓN HIJOS'''

'''
La función de Himmelblau es una función matemática bastante conocida en el campo de la 
optimización. Sus derivadas parciales son conocidas y se pueden calcular de manera exacta.
Sin embargo, si tuviéramos solo datos discretos de la función, tendríamos que usar el 
método de diferencias finitas para aproximar las derivadas.
'''
def gradient(ch, epsilon=1e-5):
    x, y = ch[0], ch[1]
    # Derivadas parciales de los genes x e y 
    grad_x = (himmelblau([x + epsilon, y]) - himmelblau([x, y])) / epsilon
    grad_y = (himmelblau([x, y + epsilon]) - himmelblau([x, y])) / epsilon
    
    return grad_x, grad_y


'''
    Aplica una mutación en un individuo basado en el gradiente descendente. La mutación
    ajusta los valores de x e y del individuo en la dirección opuesta al gradiente de la función
    de Himmelblau.
'''
def mutate_with_gradient(individual, pmut, learning_rate=0.1, epsilon=1e-5):
    if random.random() < pmut:
        # Gradientes
        grad_x, grad_y = gradient(individual, epsilon)
        
        # actualizacion en la dirección opuesta al gradiente x_t + 1 = x_t - alpha * gradiente
        new_x = individual[0] - learning_rate * grad_x
        new_y = individual[1] - learning_rate * grad_y

        new_x = max(rang[0], min(rang[1], new_x))
        new_y = max(rang[0], min(rang[1], new_y))
        
        return [new_x, new_y]

    # Si no se muta, se mantiene igual
    return individual


'''
    La mutación de los individuos es realizada mediante el gradiente descendente, refinando las soluciones.
'''
def evolve(pop, fit, pmut, ngen=100, T=2, trace=0, pcross=0.7, elitism=False):

    pop, _ = sort_pop(pop, fit)

    for i in range(ngen):
        new_poblacion = []

        if(elitism):
            best_indv = pop[0]
            new_poblacion.append(best_indv.copy())
            
        while len(new_poblacion) < len(pop):
            parent_1 = select(pop, T)
            parent_2 = select(pop, T)

            child_1, child_2 = crossover(parent_1, parent_2, pcross)

            # Usar mutación basada en el gradiente
           
            child_1 = mutate_with_gradient(child_1, pmut)
            child_2 = mutate_with_gradient(child_2, pmut)

            new_poblacion.extend([child_1, child_2])
        
        pop, fitness = sort_pop(new_poblacion[:len(pop)], fit)

        if i % trace == 0 and trace > 0:
            print(f"Generacion {i}: mejor fitness [{fitness[0]}]")
    
    print(f"Generacion {ngen}: mejor fitness [{fitness[0]}]")

    return pop

In [39]:
#Visualizamos el resultado
population = create()
population = evolve(population, fit=fitness_himmel, pmut=0.1, ngen=1000, T=3, trace=100, pcross=0.7, elitism=True)
best_individual=population[0]
himmel_value = himmelblau(best_individual)

print(f"El mejor individuo es {best_individual}")
print(f"Valor de la función de Himmelblau en ese individuo: {himmel_value:.10f}")
print(f"Valor de la función de fitness en ese individuo: {fitness_himmel(best_individual)}")

Generacion 0: mejor fitness [0.49818474402197926]
Generacion 100: mejor fitness [0.9999999999999747]
Generacion 200: mejor fitness [1.0]
Generacion 300: mejor fitness [1.0]
Generacion 400: mejor fitness [1.0]
Generacion 500: mejor fitness [1.0]
Generacion 600: mejor fitness [1.0]
Generacion 700: mejor fitness [1.0]
Generacion 800: mejor fitness [1.0]
Generacion 900: mejor fitness [1.0]
Generacion 1000: mejor fitness [1.0]
El mejor individuo es [3.0000000002907234, 2.000000001026053]
Valor de la función de Himmelblau en ese individuo: 0.0000000000
Valor de la función de fitness en ese individuo: 1.0
